In [1]:
try:
  %tensorflow_version 2.x  # Colab only.
except Exception:
  pass

import tensorflow as tf
print(tf.__version__)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # Colab only.`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.
2.6.0


In [2]:
# additional imports

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, GlobalMaxPooling2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model

In [3]:
# Load in the data
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = y_train.flatten(), y_test.flatten()
print("x_train.shape:", x_train.shape)
print("y_train.shape", y_train.shape)

170508288/170498071 [==============================] - 6s 0us/step
x_train.shape: (50000, 32, 32, 3)
y_train.shape (50000,)


In [4]:
# number of classes
K = len(set(y_train))
print("number of classes:", K)

number of classes: 10


In [5]:
# Build the model using the functional API
def create_model():
  i = Input(shape=x_train[0].shape)

  x = Conv2D(32, (3, 3), activation='relu', padding='same')(i)
  x = BatchNormalization()(x)
  x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)

  x = Flatten()(x)
  x = Dropout(0.2)(x)
  x = Dense(1024, activation='relu')(x)
  x = Dropout(0.2)(x)
  x = Dense(K, activation='softmax')(x)

  model = Model(i, x)
  return model

In [6]:
strategy = tf.distribute.MirroredStrategy()
# strategy = tf.distribute.experimental.CentralStorageStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [7]:
print(f'Number of devices: {strategy.num_replicas_in_sync}')

Number of devices: 1


In [8]:
with strategy.scope():
  model = create_model()

  model.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

In [9]:
# Fit
r = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5)

Epoch 1/5
1563/1563 [==============================] - 336s 212ms/step - loss: 1.2648 - accuracy: 0.5649 - val_loss: 1.0095 - val_accuracy: 0.6542
Epoch 2/5
1563/1563 [==============================] - 330s 211ms/step - loss: 0.8272 - accuracy: 0.7140 - val_loss: 0.9621 - val_accuracy: 0.6807
Epoch 3/5
1563/1563 [==============================] - 330s 211ms/step - loss: 0.6799 - accuracy: 0.7650 - val_loss: 0.6771 - val_accuracy: 0.7691
Epoch 4/5
1563/1563 [==============================] - 330s 211ms/step - loss: 0.5783 - accuracy: 0.8016 - val_loss: 0.5839 - val_accuracy: 0.8021
Epoch 5/5
1563/1563 [==============================] - 331s 212ms/step - loss: 0.4871 - accuracy: 0.8337 - val_loss: 0.5939 - val_accuracy: 0.8007


In [10]:
50000/391

127.8772378516624

In [11]:
10000/79

126.58227848101266

In [12]:
# Compare this to non-distributed training
model2 = create_model()
model2.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
r = model2.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5)

Epoch 1/5
1563/1563 [==============================] - 332s 212ms/step - loss: 1.2486 - accuracy: 0.5676 - val_loss: 1.1631 - val_accuracy: 0.5875
Epoch 2/5
1563/1563 [==============================] - 330s 211ms/step - loss: 0.8315 - accuracy: 0.7133 - val_loss: 1.0332 - val_accuracy: 0.6498
Epoch 3/5
1563/1563 [==============================] - 330s 211ms/step - loss: 0.6940 - accuracy: 0.7621 - val_loss: 0.6765 - val_accuracy: 0.7665
Epoch 4/5
1563/1563 [==============================] - 330s 211ms/step - loss: 0.5875 - accuracy: 0.7989 - val_loss: 0.6572 - val_accuracy: 0.7781
Epoch 5/5
1563/1563 [==============================] - 330s 211ms/step - loss: 0.4941 - accuracy: 0.8294 - val_loss: 0.6667 - val_accuracy: 0.7747
